In [1]:
import requests
import pandas as pd
import numpy as np

!pip install folium==0.5.0  
import folium 

# !conda install -c conda-forge geocoder --yes  
# import geocoder

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Use the requests library to download the webpage

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html_data= requests.get(url).text

Parse the html data using beautiful_soup.

In [3]:
soup = BeautifulSoup(html_data, 'html5lib')

Using beautiful soup extract the table and store it into a dataframe named toronto_data. The dataframe have columns PostalCode, Borough, Neighborhood. Fill in each variable with the correct data from the list cell.

In [4]:
toronto_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for cell in soup.find("tbody").find_all("td"):
    if cell.span.text != 'Not assigned':
        PostalCode = cell.p.text[:3]
        Borough = (cell.span.text).split('(')[0]
        Neighborhood = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        
        toronto_data = toronto_data.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)

toronto_data['Borough'] = toronto_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


get the coordinate data from csv

In [5]:
toronto_data_coord = pd.read_csv('https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv')
toronto_data_coord.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
toronto_data_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Create the merged data with coordinates

In [6]:
merged_toronto_data = pd.merge(toronto_data, toronto_data_coord, on='PostalCode' )
merged_toronto_data.head()

# check certain coordinates to understand merging is correct
# toronto_data_coord.loc[toronto_data_coord.PostalCode =='M5A'][['Latitude', 'Longitude']]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Function to repeat the explore all the neighborhoods in Manhattan

In [7]:
CLIENT_ID = 'U3OEYP4H1CZFH54N1MSBLG0YHJ5AET3BO4TJC3KWPXRTNVDW' # your Foursquare ID
CLIENT_SECRET = 'KHRSLJGTAHPYO4WJS3E42ZYTVQBULODHOEQ2XHPPK5LLR3SQ' # your Foursquare Secret
VERSION = '20210321' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called toronto_venues

In [ ]:
toronto_venues = getNearbyVenues(names=merged_toronto_data['Neighborhood'], latitudes=merged_toronto_data['Latitude'], longitudes=merged_toronto_data['Longitude'])

In [ ]:
toronto_venues.head()

Analzye each neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_top_venues = merged_toronto_data

# merge toronto_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_top_venues = toronto_top_venues.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_top_venues.head() # check the last columns!

Check NAN values on Cluster labels 

In [ ]:
check_nan = toronto_top_venues.isna()
check_nan.loc[check_nan['Cluster Labels'] == True]

Drop rows with NAN values in Cluster Labels

In [ ]:
toronto_top_venues.drop([5,45,95], inplace=True)
toronto_top_venues.shape

In [ ]:
check_nan = toronto_top_venues.isna()
check_nan.loc[check_nan['Cluster Labels'] == True]

Visualize the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_top_venues['Latitude'], toronto_top_venues['Longitude'], toronto_top_venues['Neighborhood'], toronto_top_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters